# Typescript Snippets
---

## String replacement using TS 
helpful for managing type differences with PascalCase and snake_case 

In [ ]:
type StringReplace< 
  TString extends string, 
  TToReplace extends string, 
  TReplacement extends string 
> = TString extends `${infer TPrefix}${TToReplace}${infer TSuffix}` 
  ? `${TPrefix}${TReplacement}${StringReplace<
    TSuffix, 
    TToReplace, 
    TReplacement
    >}`
  : TString;

type Result = StringReplace<"send_email_new", "_", "-">;

type CheckConditional = "send_email" extends `${infer TPrefix}${"_"}${infer TSuffix}`
  ? [TPrefix, TSuffix]
  : false;

type CheckReplacement = 
  "cool_something" extends `${infer TPrefix}${"_"}${infer TSuffix}`
    ? `${TPrefix}${"-"}${TSuffix}`
    : "cool_something"


---
## Optional Properties - Alternative strategy 
### use Omit`<T,key>` instead of `?` 
Avoid making a property optional to support two separate scenarios; doing so reduces type safety

In [ ]:
type user = {
	// making id optinal since unsaved users don't have an id yet 👎
	id?: number;
	name: string;
	email: string;
};

// instead, declare separate types. Then each scenario is fully type safe 👍
type User = {
	id: number;
	name: string;
	email: string;
};

// separate type for unsaved users
// deriving from the User type via Omit to keep types clean and lean
type UnsavedUser = Omit<User, 'id'>;

---
## Index Access Types
### Deep access of arrays and objects

In [ ]:
interface ColorVariants {
    primary: 'blue'
    secondary: 'red'
    tertiary: 'green'
}

// use index access type 👍
type PrimaryColor = ColorVariants['primary']

// with unions
type NonPrimaryColor = ColorVariants['secondary' | 'tertiary']
// with bigger unions
type EveryColor = ColorVariants[keyof ColorVariants]

// array of letters
type Letters = ['a', 'b', 'c']
// just 'a' or 'b'
type AorB = Letters [0 | 1];
// union of all array elements
type Letter = Letters[number]

// here's the cool one 🚀
interface UserRoleConfig {
    user: ['view', 'create', 'update'];
    superAdmin: ['view', 'create', 'update', 'delete'];
}

type Role = UserRoleConfig[keyof UserRoleConfig][number]

If you're getting the TypeScript error

'...expression of type string cannot be used to index...'

then simply specify that the 'expression of type string' is a key of the type of that object. For example,

```
const someObj:ObjectType = data;
const field = 'username';
```

// This gives an error

```const temp = someObj[field];```

// Solution 1: When the type of the object is known

```const temp = someObj[field as keyof ObjectType]```

// Solution 2: When the type of the object is not known

```const temp = someObj[field as keyof typeof someObj]```

---
## Generics 

leverage generics in React to create flexible and dynamic components

In [ ]:
import React from 'react'
//* goal is to pass any items, and have the id of that 
//* item propagate through to 2nd (render) function

interface TableProps<TItem> {
    // items: {id: string}[]
    items: TItem[]; // replaced with generic TItem
    // renderItem: (item: {id: string}) => React.ReactNode
    renderItem: (item: TItem) => React.ReactNode // replace with generic TItem
}

// need function syntax not arrow to avoid error
export function Table<TItem>(props: TableProps<TItem>) {
    return null;
}

const Component = () => {
    return (
        // <Table<{id: number}> // can manually specify the generic
        <Table
            items={[{ id: "1", name: 'Jim'}]}
            renderItem={(item) => (<div>{item.id}</div>)}
        ></Table>
    )
}

### Passing generics to types
10 examples from https://www.youtube.com/embed/dLPgQRbVquo

In [ ]:
// pass types to other types
type MyGenericType<TData> = {
  data: TData;
}

// type Example inherits type from the generic
type Example = MyGenericType<{
  firstName: string;
}>

type Example2 = MyGenericType<number>

export {};

### Passing types to functions

In [ ]:
// generic functions are funcs with a type helper mapped over the top, 
// which mapes the type to the return value

const fetchSomething = <TData>(url:string): Promise<TData> => {
  return fetch(url).then((res) => res.json());
}

fetchSomething<{firstName: string; lastName: string}>(
  "/api/endpoint"
).then((res) => {
  console.log(res)
})

### Passing types to Sets

In [ ]:
//* pass type params to other JS stuff, like Sets and Maps

const set = new Set<number>();

set.add(1)

set.add("some string") //! errors due to generic type requiring number type


### Inferring types passed to functions

In [ ]:
//* you don't always have to pass types to a generic function!

const addIdToObject = <TObject>(obj: TObject) => {
  return {
    ...obj,
    id: "123"
  }
}
// types are inferred from func args
const result = addIdToObject({
  firstName: "Jim",
  lastName: "Deola"
});

### Generic Constraints ⭐️

In [ ]:
//* ReturnType's type def expects a function
// type GetPromiseReturnType<T> = Awaited<ReturnType<T>>;

//* so, we add 'extends' to our generic type as a constraint as well 
type GetPromiseReturnType<T extends (...args: any) => any> = Awaited<ReturnType<T>>;
//* Awaited used for async stuff like when using 'await'

type MyResult = GetPromiseReturnType<
  () => Promise<{
    firstName: string;
    lastname: string;
  }>
>

type ErrorLine = GetPromiseReturnType<string> //! errors due to constraint

### Advanced Generic Constraints ⭐️

In [ ]:
//* sometimes you need to contrain the generic that gets passed in

//* we pass an object to function and returns an object with the key of the Tobj and value
//* Record is type helper that gives you an object with dynamic keys (as string, here) AND specified value
const getKeyWithHighestValue = <TObj extends Record<string, number>>(
  obj: TObj
): {
  key: keyof TObj;
  value: number;
} => {
  const keys = Object.keys(obj) as Array<keyof TObj>;

  let highestKey: keyof TObj = keys[0];
  let highestValue = obj[highestKey];

  for (const key of keys) {
    if (obj[key] > highestValue) {
      highestKey = key;
      highestValue = obj[key]
    }
  }
    return {
      key: highestKey,
      value: highestValue
    }
};

const finalResult = getKeyWithHighestValue({
  a: 1,
  b: 2,
  c: 3
});

const key = finalResult.key;
const value = finalResult.value;

console.log(key)

### Overriding Generics

In [ ]:
// use an assertion to override the expected type from the generic

const typedObjectKeys = <TObj extends {}>(
  obj: TObj
): Array<keyof TObj> => { // ⬅️ can remove Array<keyof TObj>
  // return Object.keys(obj) //! this errors bc TS can get confused with complex generics
  return Object.keys(obj) as Array<keyof TObj> // use 'as' keyword to override
}

const resultObj = typedObjectKeys({
  name: "Jim",
  age: 33
});

// written again, more concisely
const objKeys = <T extends object>(obj: T) => {
  return Object.keys(obj) as Array<keyof T>;
}

### Multiple Generics ⭐️

In [ ]:
//* we need to constrain TKey but still infer it
const getValue1 = <TObj, TKey extends keyof TObj>(obj: TObj, key: TKey) => {
  if (key === "bad") {
    throw Error("Don't access the bad key!")
  }
  return obj[key]
}

const resultValue = getValue1({
  a: 1,
  b: "some string"
},
"b"
);

console.log(resultValue);

### Default Generics

In [ ]:
//* we add a default type to generic type T, 
//* since there are no function arguments to infer from

const createSet = <T = string>() => {
  return new Set<T>();
}

const numberSet = createSet<number>();
const stringSet = createSet<string>();

const otherStringSet =  createSet();

### use generics to infer from 3rd party libs

In [ ]:
import {z} from "zod";
//* using zod to set schema and validate types, we can provide safety at runtime

const safeZodFetch = <TData>(
  url: string,
  schema: z.Schema<TData>    // z.Schema accepts type args
): Promise<TData> => {
  return fetch(url)
    .then((res) => res.json())
    .then(res => {
    return schema.parse(res);
  })
};

const safeZodFetchResult = safeZodFetch //<
// {
//   firstName: string; //* now we can remove these type args
//   lastName: string;  //* and result will still be inferred safely on type and runtime levels
// }
//>
(
  "/api/endpoint",
  z.object({
    firstName: z.string(),
    lastName: z.string()
  })
).then((res) => {
  console.log(res);
})

//* Note: by grouping complex typescript 'stuff' in one place, 
//* we can often avoid needing it in our pure js functions that we use with it.   

---
## Turning a module into a type

``` export type Action = "ADD_TODO" | "REMOVE_TODO" | "EDIT_TODO" ```

This is a common pattern in older Redux applications. Now, we've actually got a constants.ts file which has all of the elements of the union. They're even inferred by TypeScript

The exports are not actually inferred as strings. They're inferred as the literals "ADD_TODO", "REMOVE_TODO", and "EDIT_TODO".

/constants.ts
```
export const ADD_TODO = "ADD_TODO"
export const REMOVE_TODO = "REMOVE_TODO"
export const EDIT_TODO = "EDIT_TODO"
```

So there must be a way that we can extract this information and create the union type dynamically. Because otherwise, whenever we add a new element in constants.ts, we've also got to add it to the new place. It's just not very DRY.

In [ ]:
export type Action1 = "ADD_TODO" | "REMOVE_TODO" | "EDIT_TODO"

export type ActionModule = typeof import("./constants") 

// create a union type out of the keys of ActionModule
export type Action2 = ActionModule[keyof ActionModule]

This takes the exported keys of the ActionModule, and it sort of iterates over them. So we end up with ADD_TODO, REMOVE_TODO, and EDIT_TODO which stays in sync with our actual code.

---
### Fix Object Keys with `infer` and template literals


In [ ]:
// how to remove unnecessary 'maps' from object keys
interface ApiData {
    'maps:longitude': string;
    'maps:latitude': string;
}

type RemoveMapsFomObj<T> = {
    [K in keyof T as RemoveMaps<K>]: T[K];
};

type DesiredShape = RemoveMapsFomObj<ApiData>
// yields type DesiredShape = { longitude: string, latitude: string }

type RemoveMaps<T> = T extends `maps:${infer U}` ? U : T

---
## Typesafe React Context

In [ ]:
import react
import { createContext, useContext} from "react"

export interface UserContextType {
    name: string;
    age: number;
}

export const userContext = createContext<UserContextType | null>(null);

export const useUserContext = () => {
    const context = useContext(userContext);
    // handle errors to 
    if (!context) {
        throw new Error("useUserContext msut be used within a UserProvider!")
    }
}


---
## Intersection Observer React Hook

read more here
https://betterprogramming.pub/react-useinview-hook-intersection-observer-animations-and-multiple-refs-73c68a33b5b1

In [ ]:

import React, { useState, useEffect } from 'react';

export function useInView(refs: React.RefObject<HTMLElement>[]) {
  const [elements, setElements] = useState<{
    [key: string]: { isInView: boolean };
  }>({});

  useEffect(() => {
    const observerCallback = (entries: IntersectionObserverEntry[]) => {
      entries.forEach((entry) => {
        const name = entry.target.getAttribute('data-myProperty');
        if (!name) {
          console.warn(
            'Encountered entry with no name. You should add data-myProperty to every element passed to the isInView hook.'
          );
        } else {
          if (entry.isIntersecting) {
            setElements((prev) => {
              return {
                ...prev,
                [name]: {
                  isInView: true,
                },
              };
            });
          } else {
            setElements((prev) => ({
              ...prev,
              [name]: {
                isInView: false,
              },
            }));
          }
        }
      });
    };

    const observer = new IntersectionObserver(observerCallback);

    refs.forEach((ref) => {
      if (ref.current) {
        observer.observe(ref.current);
      }
    });

    return () => {
      observer.disconnect();
    };
  }, []);

  return elements;
}

---
### Type-Safe fetching with Zod
https://twitter.com/mattpocockuk/status/1610948469488771074?s=12&t=Ua6guZt5woaom80XYXfdxQ

In [ ]:
// client.ts

// import type { RouteMap } from "./server";

export const fetchFromBackend = <
  TRoute extends keyof RouteMap
>(
  route: TRoute,
  input: Parameters<RouteMap[TRoute]>[0]
) => {
  return fetch(route, {
    method: "POST",
    body: JSON.stringify(input),
  }).then((res) => res.json()) as ReturnType<
    RouteMap[TRoute]
  >;
};

// makeTypeSafeApiCall.ts

import { z } from "zod";

type MaybePromise<T> = T | Promise<T>;

export const makeTypeSafeApiCall = <TInput, TOutput>(
  inputSchema: z.Schema<TInput>,
  outputSchema: z.Schema<TOutput>,
  handler: (input: TInput) => MaybePromise<TOutput>
): ((input: TInput) => MaybePromise<TOutput>) => {
  return async (input: unknown) => {
    const result = await handler(inputSchema.parse(input));
    return outputSchema.parse(result);
  };
};

// server.ts

// import { z } from "zod";
// import { makeTypeSafeApiCall } from "./makeTypeSafeApiCall";

export type RouteMap = {
  "/user/create": typeof createUser;
};

export const createUser = makeTypeSafeApiCall(
  z.object({
    email: z.string(),
  }),
  z.object({
    id: z.string(),
    email: z.string(),
  }),
  async ({ email }) => {
    return {
      id: "123",
      email,
    };
  }
);

---
### Enums @ compile time

Even though Enums are real objects that exist at runtime, the keyof keyword works differently than you might expect for typical objects. Instead, use keyof typeof to get a Type that represents all Enum keys as strings.

In [ ]:
enum LogLevel {
  ERROR,
  WARN,
  INFO,
  DEBUG,
}
 
/**
 * This is equivalent to:
 * type LogLevelStrings = 'ERROR' | 'WARN' | 'INFO' | 'DEBUG';
 */
type LogLevelStrings = keyof typeof LogLevel;
 
function printImportant(key: LogLevelStrings, message: string) {
  const num = LogLevel[key];
  if (num <= LogLevel.WARN) {
    console.log("Log level key is:", key);
    console.log("Log level value is:", num);
    console.log("Log level message is:", message);
  }
}
printImportant("ERROR", "This is a message");

### Objects vs Enums
In modern TypeScript, you may not need an enum when an object with as const could suffice:

In [ ]:
const enum EDirection {
  Up,
  Down,
  Left,
  Right,
}
 
const ODirection = {
  Up: 0,
  Down: 1,
  Left: 2,
  Right: 3,
} as const;
 
EDirection.Up;
           // (enum member) EDirection.Up = 0
 
ODirection.Up;
           // (property) Up: 0
 
// Using the enum as a parameter
function walk(dir: EDirection) {}
 
// It requires an extra line to pull out the values
type Direction = typeof ODirection[keyof typeof ODirection];
function run(dir: Direction) {}
 
walk(EDirection.Left);
run(ODirection.Right);

---
Generic Types in React component

https://www.totaltypescript.com/tips/use-generics-in-react-to-make-dynamic-and-flexible-components

In [ ]:
import * as React from 'react';

// takes type of TItem and passes it it's render function
interface TableProps<TItem> {
  items:TItem[];
  renderItem: (item:TItem) => React.ReactNode;
}

//must be function component not arrow
export function Table<TItem>(props: TableProps<TItem>) {
  return null;
}

const component = () => {
  return (
    <Table<{id:number}> 
      items={[
        {id: 1,
        name: 'jim'},
      ]}
      renderItem={(item)=> <div>{item.id}</div>}
      ></Table>
  )
}

---
### Declare Globals 

Example with with common reducer pattern ie redux

In [ ]:
// /types.ts
declare global {
  interface GlobalReducerEvent {
    LOG_IN: {};
  }
}

export type GlobalReducer<TState> = (
  state: TState,
  event: {
    [EventType in keyof GlobalReducerEvent]: {
      type: EventType;
    } & GlobalReducerEvent[EventType];
  }[keyof GlobalReducerEvent]
) => TState;

// /reducer.ts

declare global {
  interface GlobalReducerEvent {
    ADD_TODO: { text: string }
  }
}

export const todosReducer: GlobalReducer<{ todos: { id: string }[] }> = (
  state,
  event
) => {
  // event.type === 'LOG_IN'
  // event.type = 'ADD_TODO'
  return state
}

export const userReducer: GlobalReducer<{ id: string }> = (
  state,
  event
) => {
  // event.type === 'LOG_IN'
  // event.type === 'ADD_TODO'
  return state
}

---
### Components as="..."


In [ ]:
import React, { ComponentProps, JSXElementConstructor } from "react";

export const Component = <
  T extends keyof JSX.IntrinsicElements | JSXElementConstructor<any>,
>(
  props: {
    as: T;
  } & ComponentProps<T>,
) => {
  const Comp = props.as;
  return <Comp {...(props as any)}></Comp>;
};

const Link = (props: { href: string; children?: React.ReactNode }) => {
  return <a href={props.href}>{props.children}</a>;
};

const yeah = <Component as={Link} href="awdawd"></Component>;

In [ ]:
export const Component = <TAs extends keyof JSX.IntrinsicElements>(
  props: {
    as: TAs;
  } & JSX.IntrinsicElements[TAs],
) => {
  const Comp = props.as as string;
  return <Comp {...(props as any)}></Comp>;
}

---
### TS + React Patterns
#### Adapter Pattern for handling heterogeneous types

more info: https://itnext.io/top-5-react-typescript-design-patterns-to-boost-your-skills-to-the-next-level-5b5f54dd934f#a542

The Adapter Pattern is a design pattern that allows two incompatible interfaces to work together. In React, this pattern can be used to adapt an existing component to work with a different data source or API.

Suppose you have a React application that fetches data from an API using the Axios library. However, for some reason, you need to switch to using the Fetch API instead of Axios. Unfortunately, the Fetch API has a different interface than Axios, which means you’ll need to rewrite all the API calls in your codebase to use the Fetch API.

Instead of rewriting all the API calls, you can use the Adapter Pattern to create an adapter that adapts the Fetch API to the same interface as Axios. You can create a fetchAdapter that takes the same arguments and returns the same shape of data as Axios. This way, you can minimize the amount of code you need to change and reduce the risk of bugs

In [ ]:
interface AxiosResponse<T = any> {
  data: T;
}

interface AxiosError<T = any> extends Error {
  response: AxiosResponse<T>;
}

interface AxiosInstance {
  get<T = any>(url: string, config?: any): Promise<AxiosResponse<T>>;
  post<T = any>(
    url: string,
    data?: any,
    config?: any
  ): Promise<AxiosResponse<T>>;
  // and other methods as needed
}

interface FetchAdapter {
  get<T = any>(url: string, config?: any): Promise<AxiosResponse<T>>;
  post<T = any>(
    url: string,
    data?: any,
    config?: any
  ): Promise<AxiosResponse<T>>;
  // and other methods as needed
}

const fetchAdapter: FetchAdapter = {
  get: async <T = any>(url: string, config?: any) => {
    const response = await fetch(url, config);
    const data = await response.json();
    return { data };
  },
  post: async <T = any>(url: string, data?: any, config?: any) => {
    const response = await fetch(url, {
      method: 'POST',
      headers: {
        'Content-Type': 'application/json',
      },
      body: JSON.stringify(data),
      ...config,
    });
    const responseData = await response.json();
    return { data: responseData };
  },
  // and other methods as needed
};

// Usage example
const axiosInstance: AxiosInstance = {
  get: async (url: string, config?: any) => {
    try {
      const response = await fetchAdapter.get(url, config);
      return response;
    } catch (error) {
      const axiosError: AxiosError = new Error();
      axiosError.response = error.response;
      throw axiosError;
    }
  },
  post: async (url: string, data?: any, config?: any) => {
    try {
      const response = await fetchAdapter.post(url, data, config);
      return response;
    } catch (error) {
      const axiosError: AxiosError = new Error();
      axiosError.response = error.response;
      throw axiosError;
    }
  },
  // and other methods as needed
};

In this example, we define an interface AxiosInstance that describes the interface of Axios. We also define an interface FetchAdapter that describes the interface we want to use for the Fetch API. We then define an object fetchAdapter that implements the FetchAdapter interface by adapting the Fetch API to the Axios interface. Finally, we create an instance of AxiosInstance that uses the fetchAdapter to make API calls.

#### Facade Pattern for encapsulating complex components

The Facade Pattern is a design pattern that provides a simplified interface to a complex system. In React, this pattern can be used to create a simpler API for complex components, making it easier for other developers to use those components.

Suppose you have a complex API that fetches data from multiple sources and aggregates it to display a chart. The API has several methods, each with multiple parameters, and requires a lot of configuration to work correctly.

To make it easier for other developers to use this API, you can create a Facade component that provides a simplified interface. The Facade component can handle all the complexity behind the scenes and expose a simple, easy-to-use API for other components to consume.

In [ ]:
import React, { useState, useEffect } from 'react';
import ChartApi from './ChartApi';

type FacadeProps = {
  chartType: string;
  dataUrl: string;
  width: number;
  height: number;
};

const ChartFacade: React.FC<FacadeProps> = ({
  chartType,
  dataUrl,
  width,
  height,
}) => {
  const [data, setData] = useState([]);

  useEffect(() => {
    const fetchData = async () => {
      const chartApi = new ChartApi();
      const rawData = await chartApi.fetchData(dataUrl);
      const chartData = chartApi.processData(rawData);
      setData(chartData);
    };
    fetchData();
  }, [dataUrl]);

  return <Chart type={chartType} data={data} width={width} height={height} />;
};

export default ChartFacade;

---
### Generic Compound React Component
https://codesandbox.io/embed/young-dream-ihjcq3?file=/src/App.tsx&codemirror=1

In [ ]:
import React from "react";
import "./styles.css";

type Fruit = "apple" | "banana";

// set of components that share the same API
type FruitProps = {
  name: Fruit;
};

const COMPONENTS: Record<Fruit, React.ComponentType<FruitProps>> = {
  apple: ({ name }) => <div>{name}: 🍏</div>,
  banana: ({ name }) => <div>{name}: 🍌</div>
};

export default function App() {
  const [fruit, setFruit] = React.useState<Fruit>("apple");
  const FruitComponent = COMPONENTS[fruit];
  return (
    <div>
      <FruitComponent name={fruit} />
      <button
        onClick={() => {
          setFruit((fruit) => (fruit === "apple" ? "banana" : "apple"));
        }}
      >
        toggle
      </button>
    </div>
  );
}

---
### Obtaining keys of deeply nest object

only works for objects with a finite depth. If we have an object with an infinite depth, such as an object that contains a reference to itself, the type definition will result in a stack overflow error.

In [ ]:
const obj = {
  a: {
    b: 1,
    c: {
      d: 2,
      e: 3,
    },
  },
  f: {
    g: 4,
  },
  h: undefined,
};

type DeepKeys<T> = T extends object
  ? {
      [K in keyof T]-?: K extends string | number
        ? `${K}` | `${K}.${DeepKeys<T[K]>}`
        : never;
    }[keyof T]
  : never;

function getAllKeys<T extends object>(obj: T): DeepKeys<T>[] {
  return Object.keys(obj) as DeepKeys<T>[];
}

const keys = getAllKeys(obj);
console.log(keys); // ["a" | "f" | "h" | "a.b" | "a.c" | "a.c.d" | "a.c.e" | "f.g"]

---
### Returning Never with nullish coelescense

Tired of writing if statements every time you need to check if something's defined?

Make a little 'raise' function to throw an error, and inline it with a nullish coalescing operator.

In [ ]:
// this fn will never return, just throw and error
const raise = (err: string): never => {
  throw new Error(err);
};

// we can now use raise to ensure 'id' is provided to params
const Page = (props: { params: { id?: string;}}) => {
  const id = props.params.id ?? raise('No id provided!');
};

---
### Next.js typed response

lost the source file 

In [ ]:
type NextApiRequest = { body: any }
type NextApiResponse = { json: (arg: unknown) => void }

const getSessionUserId = (): number | null => {
  return Math.random() || null
}

const parseNextApiRequestBody = <B = object>(
  request: NextApiRequest
): Partial<B> | null => {
  try {
    const parsedBody = JSON.parse(request.body as string) as unknown
    return typeof parsedBody === 'object' ? parsedBody : null
  } catch {
    return null
  }
}

type Expand<T> = T extends ((...args: any[]) => any) | Date | RegExp
  ? T
  : T extends ReadonlyMap<infer K, infer V>
  ? Map<Expand<K>, Expand<V>>
  : T extends ReadonlySet<infer U>
  ? Set<Expand<U>>
  : T extends ReadonlyArray<unknown>
  ? `${bigint}` extends `${keyof T & any}`
    ? { [K in keyof T]: Expand<T[K]> }
    : Expand<T[number]>[]
  : T extends object
  ? { [K in keyof T]: Expand<T[K]> }
  : T

type Options = {
  requiresAuthentication?: boolean
  parseBody?: boolean
}

type CallbackOptions<B = never, O extends Options = Options> = {
  request: NextApiRequest
  response: NextApiResponse
} & (O extends { requiresAuthentication: true } ? { userId: string } : object) &
  (O extends { parseBody: true } ? { parsedRequestBody: B } : object)

const handleRequest =
  <O extends Options>(options: O) =>
  <B = never>(callback: (options: CallbackOptions<B, O>) => Promise<void>) =>
  async (request: NextApiRequest, response: NextApiResponse) => {
    // If the user is not found, we can return a response right away.
    const userId = getSessionUserId()

    if (options.requiresAuthentication && !userId) {
      return void response.json({ error: 'missing authentication' })
    }

    // Check if the request's body is valid.
    const parsedRequestBody = options.parseBody
      ? parseNextApiRequestBody(request)
      : undefined

    if (options.parseBody && !parsedRequestBody) {
      return void response.json({ error: 'invalid payload' })
    }

    return callback({
      request,
      response,
      ...(options.parseBody ? { parsedRequestBody } : {}),
      ...(options.requiresAuthentication ? { userId } : {}),
    } as CallbackOptions<B, O>)
  }

const handlerWithBodyType = handleRequest({ parseBody: true })<{
  hello: string
}>(async (options) => {
  // `options.parsedRequestBody` is of type `{ hello: string }`
  void typeof options?.parsedRequestBody
  //                   ^?
})

const handlerWithAuth = handleRequest({ requiresAuthentication: true })(
  async (options) => {
    // `options` contains `request` and `response` and `userId` and not `parsedRequestBody` as expected.
    const expandedOptions: Expand<typeof options> = options
    //    ^?
  }
)


---
### Type utilities

from react-typescript-tutorial by Matt Pocock
https://github.com/total-typescript/react-typescript-tutorial/blob/main/src/helpers/type-utils.ts


In [ ]:
export type Expect<T extends true> = T;
export type ExpectTrue<T extends true> = T;
export type ExpectFalse<T extends false> = T;
export type IsTrue<T extends true> = T;
export type IsFalse<T extends false> = T;

export type Equal<X, Y> = (<T>() => T extends X ? 1 : 2) extends <
  T,
>() => T extends Y ? 1 : 2
  ? true
  : false;
export type NotEqual<X, Y> = true extends Equal<X, Y> ? false : true;

// https://stackoverflow.com/questions/49927523/disallow-call-with-any/49928360#49928360
export type IsAny<T> = 0 extends 1 & T ? true : false;
export type NotAny<T> = true extends IsAny<T> ? false : true;

export type Debug<T> = { [K in keyof T]: T[K] };
export type MergeInsertions<T> = T extends object
  ? { [K in keyof T]: MergeInsertions<T[K]> }
  : T;

export type Alike<X, Y> = Equal<MergeInsertions<X>, MergeInsertions<Y>>;

/**
 * Expect that one type is assignable to another.
 *
 * @example
 *
 * type tests = [
 *   // Expect that `number` is assignable to `1`.
 *   Expect<Extends<1, number>>,
 *   // Expect that `abc` is assignable to `string`
 *   Expect<Extends<'abc', string>>,
 * ];
 */
export type Extends<VALUE, EXPECTED> = EXPECTED extends VALUE ? true : false;
export type ExpectValidArgs<
  FUNC extends (...args: any[]) => any,
  ARGS extends any[],
> = ARGS extends Parameters<FUNC> ? true : false;

export type UnionToIntersection<U> = (
  U extends any ? (k: U) => void : never
) extends (k: infer I) => void
  ? I
  : never;

---
### Iterating over objects

https://oida.dev/typescript-iterating-over-objects/


In [ ]:
type Person = {
  name: string,
  age: number
}


function printPerson(p: Person) {
  const you: Person = {
    name: "Reader",
    age: NaN
  };

  Object.keys(p).forEach((k) => {
    console.log(k, you[k])
  })  
}

const me = {
  name: "Stefan",
  age: 40,
  website: "https://fettblog.eu"
}

printPerson(me);

// If Object.keys(p) returns an array of type keyof Person[], you will be able to access
// other objects of Person, too. This might not add up. In our example, we just print undefined. 
// But what if you try to do something with those values. This will break at runtime.

//TypeScript prevents you from scenarios like this. It’s honest and says: Well, you think it might
// be keyof Person, but in reality, it can be so much more.

//Only type guards can help you:

function isKey<T>(x: T, k: PropertyKey): k is keyof T {
  return k in x
}

function printPerson2(p: Person) {
  Object.keys(p).forEach((k) => {
      if(isKey(p, k)) console.log(k, p[k]) // All fine!
  })
}

// With For/in loops and generics, typescript can have much more effectie type narrowing

function printPerson3<T extends Person>(p: T) {
  for (let k in p) {
    console.log(k, p[k]) // This works
  }
}

### Unique Array Utility Type

https://ja.nsommer.dk/articles/type-checked-unique-arrays.html


In [ ]:
type InArray<T, X> =
  // See if X is the first element in array T
  T extends readonly [X, ...infer _Rest]
    ? true
    // If not, is X the only element in T?
    : T extends readonly [X]
      ? true
      // No match, check if there's any elements left in T and loop recursive
      : T extends readonly [infer _, ...infer Rest]
        ? InArray<Rest, X>
        // There's nothing left in the array and we found no match
        : false
        
type UniqueArray<T> =
  T extends readonly [infer X, ...infer Rest]
    // We've just extracted X from T, having Rest be the remaining values.
    // Let's see if X is in Rest, and if it is, we know we have a duplicate
    ? InArray<Rest, X> extends true
      ? ['Encountered value with duplicates:', X]
      // X is not duplicated, move on to check the next value, and see
      // if that's also unique.
      : readonly [X, ...UniqueArray<Rest>]
    // T did not extend [X, ...Rest], so there's nothing to do - just return T
    : T

---
### Generic Fetch Function by Matt Pocock (twitter post)

https://www.typescriptlang.org/play?#code/KYDwDg9gTgLgBAYwgOwM7wObHgXjgQ1QE9kE4AKAKDjgFcoAbALjnSgEtkMAaauTsLRgsASsCRQAJgB42nHqxgcuAPkoBKODhVwA3nyjZ6yOADNsCABZUaNAAYASXfQYBfAPxPkwAO5wAqiIAMgDKwPhQVgAKEfgAtqjkAkLqAHQwECFK8uTqrnZ86gDclK4llKCQsIgo6HCQdXiEJGQ2dIwscly8NMnCcGISMl0KI2qa2noGRlAm5jBW5C7cU7ZwcdiWEJIsAERRAPIhACq7PbYARttELABSIQcAcqkj7KZESciCMOrnrsWlcowIhgYBwADChnwMGAAEEogBJACym22Wjg0j4xwRXyEcFAMOQklQA3E0GG2W6imUGBU52OByE30oKnIEDAMFQLH0NBcnUpGBKNA2MC2OzguwA4gBRU5wAA+EsOJ12JX+Wh0n2+LGxuJ+GrgUSgEDi7FQwGkDKZQhU5SQaHgCChMPhyNR4sh4RdiJRorROD4bI5qAmmr6odWtntdRFYvR7M5qVj-pweClst2cHccCw8BYDRg5TWhhgxgofDWyckQcTy34evUFbWcAA9C24Ai4vgMJwwcAoMaoHBLPgiQx5MP+8BUjOm2t0pZgMhyORDCGDWvUgArVAoXKNtYAmhlSiUFsAKnP1HPcAAmhBaIhR3BDLRzXBRWazLRSDB2CgCBJLtkCID92A2ElCDgIgH2vOBxwAazBDJEGdMF8AYBhoIfIdXXWd1UAAQmvFtKGjTBsH8c0hzwJ0vThH13UxGhdH4cURjgVwVjbOBjkXetmRYuBkHiYB+RpTjW3bPiwQfGBmVZHl8L9cV01Oc4+QlFs337M5SgBShcyo-tyFY9hxQARgAJgAZk44opLgWFMI-EEwVQfBzGIoA

In [ ]:
export const get = async (
  url: string,
  input: Record<string, string>
) => {
  return fetch(
    `${url}?${new URLSearchParams(input).toString()}`
  );
};

export const post = async (
  url: string,
  input: Record<string, string>
) => {
  return fetch(url, {
    method: "POST",
    body: JSON.stringify(input),
  });
};

type CreateAPIMethod = <
  TInput extends Record<string, string>,
  TOutput
>(opts: {
  url: string;
  method: "GET" | "POST";
}) => (input: TInput) => Promise<TOutput>;

const createAPIMethod: CreateAPIMethod =
  (opts) => (input) => {
    const method = opts.method === "GET" ? get : post;

    return (
      method(opts.url, input)
        // Imagine error handling here...
        .then((res) => res.json())
    );
  };

/**
 * You can reuse this function as many times as you
 * like to create all your API methods!
 */
const getUser = createAPIMethod<
  { id: string }, // The input
  { name: string } // The output
>({
  method: "GET",
  url: "/user",
});

getUser({ id: 123 }); // All type safe!


---
### AmesDean's custom Axios Hook
(work in progress)

In [ ]:
import axios, { AxiosRequestConfig, AxiosError, AxiosResponse } from "axios";
import { useReducer, useState, useEffect } from "react";

// Define default configuration for the Axios instance
const defaultConfig: AxiosRequestConfig = {
  baseURL: "", // Replace with your API base URL
  timeout: 10000, // Default timeout
  // method: "GET", // Default HTTP method
  headers: {
    "Content-Type": "application/json", // Default content type
  },
};

// Create an Axios instance with the default configuration
const axiosInstance = axios.create(defaultConfig);

// Request Interceptor
axiosInstance.interceptors.request.use(
  (config) => {
    // Modify or log the request config here
    return config;
  },
  (error) => {
    // Handle request error here
    return Promise.reject(error);
  },
);

// Response Interceptor
axiosInstance.interceptors.response.use(
  (response) => {
    // Modify or log the response here
    return response;
  },
  (error: AxiosError) => {
    // Handle response error here
    return Promise.reject(error);
  },
);

// Enhanced TypeScript types for the Axios React hook with mapped types
export interface AxiosRequestConfigExtended<T = unknown>
  extends AxiosRequestConfig {
  transformResponseData?: (data: any) => T;
}

export type ApiRequest<T> = {
  endpoint: string;
  config?: AxiosRequestConfigExtended<T>;
};

export interface MappedApiResponse<T> {
  data: T | null;
  loading: boolean;
  error: MappedAxiosError | null;
}

export type MappedAxiosError = {
  message: string;
  code?: string;
  response?: {
    data: any;
    status: number;
  };
};

// Callback types for external logic
type RequestCallback = (config: AxiosRequestConfig) => void;
type ResponseCallback = (response: AxiosResponse) => void;
type ErrorCallback = (error: AxiosError) => void;

// Define action types for reducer
type ActionType<T> =
  | { type: "FETCH_INIT" }
  | { type: "FETCH_SUCCESS"; payload: T }
  | { type: "FETCH_FAILURE"; payload: MappedAxiosError };

// Reducer function to handle state changes
function reducer<T>(
  state: MappedApiResponse<T>,
  action: ActionType<T>,
): MappedApiResponse<T> {
  switch (action.type) {
    case "FETCH_INIT":
      return { ...state, loading: true, error: null };
    case "FETCH_SUCCESS":
      return { ...state, loading: false, data: action.payload, error: null };
    case "FETCH_FAILURE":
      return { ...state, loading: false, data: null, error: action.payload };
    default:
      return state;
  }
}

// generic type T is used to define the type of the data we are expecting from the API
// AxiosRequestConfigExtended is used to extend the AxiosRequestConfig type
// with our custom transformResponseData function
// The useAxios hook returns a state object with data, loading and error properties
export function useAxios<T = unknown>({
  endpoint,
  config,
  onRequestCallback,
  onResponseCallback,
  onErrorCallback,
}: ApiRequest<T> & {
  onRequestCallback?: RequestCallback;
  onResponseCallback?: ResponseCallback;
  onErrorCallback?: ErrorCallback;
}): MappedApiResponse<T> {
  // Use useReducer for state management
  const [state, dispatch] = useReducer(
    (state: MappedApiResponse<T>, action: ActionType<T>) =>
      reducer<T>(state, action),
    {
      data: null,
      loading: false,
      error: null,
    },
  );

  useEffect(() => {
    // Internal request interceptor
    const requestInterceptor = axiosInstance.interceptors.request.use(
      (config) => {
        onRequestCallback?.(config);
        return config;
      },
      (error) => {
        onErrorCallback?.(error);
        return Promise.reject(error);
      },
    );

    // Internal response interceptor
    const responseInterceptor = axiosInstance.interceptors.response.use(
      (response) => {
        onResponseCallback?.(response);
        return response;
      },
      (error) => {
        onErrorCallback?.(error);
        return Promise.reject(error);
      },
    );

    // Check if the method is GET or POST
    // const isGetMethod = config?.method?.toUpperCase() === "GET";
    // const isPostMethod = config?.method?.toUpperCase() === "POST";

    const fetchData = async () => {
      dispatch({ type: "FETCH_INIT" });
      try {
        // Additional logic based on the method can be implemented here

        const result = await axiosInstance(endpoint, config);
        dispatch({
          type: "FETCH_SUCCESS",
          payload: config?.transformResponseData
            ? config.transformResponseData(result.data)
            : result.data,
        });
      } catch (error) {
        const axiosError = error as AxiosError;
        dispatch({
          type: "FETCH_FAILURE",
          payload: {
            message: axiosError.message,
            code: axiosError.code,
            response: axiosError.response
              ? {
                  data: axiosError.response.data,
                  status: axiosError.response.status,
                }
              : undefined,
          },
        });
      }
    };

    fetchData();

    // Eject interceptors on cleanup
    return () => {
      axiosInstance.interceptors.request.eject(requestInterceptor);
      axiosInstance.interceptors.response.eject(responseInterceptor);
    };
  }, [
    endpoint,
    config,
    onRequestCallback,
    onResponseCallback,
    onErrorCallback,
  ]);

  return state;
}

//* Example component using the useAxios hook with all parameters

// import React from 'react';
// import { AxiosRequestConfig, AxiosResponse, AxiosError } from 'axios';
// import { useAxios } from '../hooks/useAxios';
// import { Product } from '../types/Product'; // Assuming a Product type is defined

// const ProductList: React.FC = () => {
//     // Custom callback functions
//     const handleRequest = (config: AxiosRequestConfig) => {
//         console.log('Request made with config:', config);
//         // Modify the config or add custom logic here if needed
//     };

//     const handleResponse = (response: AxiosResponse) => {
//         console.log('Response received:', response);
//         // Process response data or add custom logic here
//     };

//     const handleError = (error: AxiosError) => {
//         console.error('An error occurred:', error);
//         // Handle the error or add custom logic here
//     };

//     // Using the useAxios hook with all parameters
//     const { response: products, error, loading } = useAxios<Product[]>({
//         endpoint: '/api/products',
//         config: { method: 'GET' },
//         onRequestCallback: handleRequest,
//         onResponseCallback: handleResponse,
//         onErrorCallback: handleError,
//     });

//     // Render loading state
//     if (loading) return <p>Loading products...</p>;

//     // Render error state
//     if (error) return <p>Error fetching products: {error.message}</p>;

//     // Render product data
//     return (
//         <div>
//             <h2>Product List</h2>
//             <ul>
//                 {products && products.map(product => (
//                     <li key={product.id}>{product.name}</li> // Assuming each product has 'id' and 'name'
//                 ))}
//             </ul>
//         </div>
//     );
// };

// export default ProductList;


#### AmesDean's custom error class
(work in progress)

In [ ]:
//* Axios Types
import axios, { AxiosRequestConfig } from "axios";

export interface CustomAxiosResponse<T> {
	data: T;
	status: number;
	statusText: string;
	headers: Record<string, string>;
	config: AxiosRequestConfig;
	request?: any;
}

export interface CustomAxiosRequest<T> extends AxiosRequestConfig {
	data?: T;
}

//* example usage
async function makeRequest<T>(
	config: CustomAxiosRequest<T>
): Promise<CustomAxiosResponse<T>> {
	try {
		const response = await axios(config);
		return response.data;
	} catch (error) {
		throw error;
	}
}

//* Custom Error Class
import axios, { AxiosError } from "axios";
import { CustomAxiosRequest, CustomAxiosResponse } from "./Axios";

//* Error interfaces
interface BaseError<T = any> extends Error {
	data?: T;
}
interface AuthenticationError<T = any> extends BaseError<T> {}
interface ValidationError<T = any> extends BaseError<T> {}
interface NotFoundError<T = any> extends BaseError<T> {}

//! errors using arrow functions (not using)
// const BaseError = <T = any>(message: string, data?: T): BaseError => {
// 	const error = new Error(message) as Error & { data?: T };
// 	error.data = data;
// 	return error;
// };

// const AuthenticationError = <T = any>(
// 	message: string,
// 	data?: T
// ): AuthenticationError => BaseError<T>(message, data);

// const NotFoundError = <T = any>(message: string, data?: T): NotFoundError =>
// 	BaseError<T>(message, data);

// const ValidationError = <T = any>(message: string, data?: T): ValidationError =>
// 	BaseError<T>(message, data);
//!

// BaseError class
class BaseError<T = any> extends Error {
	data?: T;

	constructor(message: string, data?: T) {
		super(message);
		this.data = data;
		Object.setPrototypeOf(this, BaseError.prototype);
	}
}

// AuthenticationError class
class AuthenticationError<T = any> extends BaseError<T> {
	constructor(message: string, data?: T) {
		super(message, data);
		Object.setPrototypeOf(this, AuthenticationError.prototype);
	}
}

class ValidationError<T = any> extends BaseError<T> {
	constructor(message: string, data?: T) {
		super(message, data);
		Object.setPrototypeOf(this, ValidationError.prototype);
	}
}

class NotFoundError<T = any> extends BaseError<T> {
	constructor(message: string, data?: T) {
		super(message, data);
		Object.setPrototypeOf(this, ValidationError.prototype);
	}
}

//! Axios Request wrapper function with error handling
export const superRequestHandler = <T>(requestFunction: () => Promise<T>) => {
	return function axiosRequestWithHandling<T>(
		requestFunction: (
			config: CustomAxiosRequest<T>
		) => Promise<CustomAxiosResponse<T>>
	) {
		try {
			return async function (config: CustomAxiosRequest<T>): Promise<T> {
				try {
					const response = await requestFunction(config);
					return response.data;
				} catch (error) {
					if (axios.isAxiosError(error)) {
						const { response } = error as AxiosError<CustomAxiosResponse<T>>;

						if (response) {
							const { status, data } = response;

							if (status === 401) {
								throw new AuthenticationError<typeof data>(
									"Authentication failed.",
									data
								);
							} else if (status === 404) {
								throw new NotFoundError<typeof data>(
									"Failed to find data.",
									data
								);
							} else if (status === 422) {
								throw new ValidationError<typeof data>(
									"Validation failed.",
									data
								);
							}
						}
					}

					throw new BaseError("An unknown error occurred.");
				}
			};
		} catch (error) {
			if (error instanceof AuthenticationError) {
				console.error("Authentication Error:", error.message, error.data);
			} else if (error instanceof ValidationError) {
				console.error("Validation Error:", error.message, error.data);
			} else if (error instanceof NotFoundError) {
				console.error("Validation Error:", error.message, error.data);
			} else {
				if (axios.isAxiosError(error)) {
					const { response } = error as AxiosError<CustomAxiosResponse<T>>;
					if (response) {
						const { status, data } = response;
						console.error(`Unknown Error: ${status}:`, error.message, data);
					}
				}
			}
		}
	};
};


---
### Route Typing in React

In [ ]:
const routes = {
  user: ["get-user", "get-all-users"],
  comment: ["get-comment", "get-all-comments"],
} as const;

type Routes = typeof routes;

type PossibleUrl = {
  [K in keyof Routes]: `/${K}/${Routes[K][number]}`;
}[keyof Routes];

type Example = Record<PossibleUrl, {}>;
//   ^?



---
### recursive typing solution for a number 1 - 100

ie instead of 

`type AnyNumber = 1 | 2 | 3 | 4 ...`

In [ ]:
type _NumbersBefore<N extends number, A extends number[] = []> = A['length'] extends N
  ? A[number]
  : _NumbersBefore<N, [...A, A['length']]>;

type NumbersBefore<N extends number> = _NumbersBefore<N>;

type NumbersInRange<A extends number, B extends number> = Exclude<NumbersBefore<B>, NumbersBefore<A>>;

type To100 = NumbersBefore<101>;

type From50To100 = NumbersInRange<50, 101>;


---
### Fetch With Zod Schema

In [ ]:
declare const fetchWithSchema: <T>(
  url: string,
  // 2. Declaring only what we need is much
  // simpler, and drops the zod dependency
  schema: {
    parse: (value: unknown) => T;
  }
) => Promise<T>;

import { z } from "zod";

fetchWithSchema(
  "https://example.com/api/user",
  z.object({
    id: z.number(),
  })
).then((res) => {
  // 3. Still works!
  console.log(res);
  //          ^?
});


### API Methods

In [ ]:
export const get = async (
  url: string,
  input: Record<string, string>
) => {
  return fetch(
    `${url}?${new URLSearchParams(input).toString()}`
  );
};

export const post = async (
  url: string,
  input: Record<string, string>
) => {
  return fetch(url, {
    method: "POST",
    body: JSON.stringify(input),
  });
};

type CreateAPIMethod = <
  TInput extends Record<string, string>,
  TOutput
>(opts: {
  url: string;
  method: "GET" | "POST";
}) => (input: TInput) => Promise<TOutput>;

const createAPIMethod: CreateAPIMethod =
  (opts) => (input) => {
    const method = opts.method === "GET" ? get : post;

    return (
      method(opts.url, input)
        // Imagine error handling here...
        .then((res) => res.json())
    );
  };

/**
 * You can reuse this function as many times as you
 * like to create all your API methods!
 */

const getUser = createAPIMethod<
  { id: string }, // The input
  { name: string } // The output
>({
  method: "GET",
  url: "/user",
});

getUser({ id: 123 }); // All type safe!



---

Omit and Pick are some of the most loved utility types in TypeScript. They let you create new types by excluding or selecting specific properties from an existing type.

```
type Album = {
  id: string;
  title: string;
  genre: string;
};

type AlbumWithoutId = Omit<Album, "id">;

// The album
const album: AlbumWithoutId = {
  id: "1",
//Object literal may only specify known properties, and 'id' does not exist in type 'AlbumWithoutId'.

  title: "The Dark Side of the Moon",
  genre: "Progressive Rock",
};
```

However, they have some pretty odd behavior when used with union types.

The Problem

Consider a scenario where we have three types for Album, CollectorEdition, and DigitalRelease.

These types share two common properties - id and title - but each also one unique attribute each:
```
type Album = {
  id: string; // same per type
  title: string; // same per type
  genre: string; // different
};

type CollectorEdition = {
  id: string; // same per type
  title: string; // same per type
  limitedEditionFeatures: string[]; // different
};

type DigitalRelease = {
  id: string; // same per type
  title: string; // same per type
  digitalFormat: string; // different
};
```

After creating a MusicProduct type that is a union of these three types, say we want to create a MusicProductWithoutId type, mirroring the structure of MusicProduct but excluding the id field:
```
type MusicProduct = Album | CollectorEdition | DigitalRelease;

type MusicProductWithoutId = Omit<MusicProduct, "id">;
              
type MusicProductWithoutId = {
    title: string;
}
```

You might assume that MusicProductWithoutId would be a union of the three types minus the id field. However, what we get instead is a simplified object type containing only title – the other properties that were shared across all types, without id.
```
// Expected:
type MusicProductWithoutId1 =
  | Omit<Album, "id">
  | Omit<CollectorEdition, "id">
  | Omit<DigitalRelease, "id">;

// Actual:
type MusicProductWithoutId2 = {
  title: string;
};
```

This is particularly annoying given that Partial and Required work as expected with union types:

```
type PartialMusicProduct = Partial<MusicProduct>;
             
type PartialMusicProduct = Partial<Album> | Partial<CollectorEdition> | Partial<DigitalRelease>
```

This stems from how Omit processes union types. Rather than iterating over each union member, it combines them into a single structure it can understand.

The technical reason for this is that Omit and Pick are not distributive. This means that when you use them with a union type, they don't operate individually on each union member.

The Solution: DistributiveOmit and DistributivePick

In order to address this, we can create a DistributiveOmit type. It's defined similarly to Omit but operates individually on each union member.

```
type DistributiveOmit<T, K extends PropertyKey> = T extends any
  ? Omit<T, K>
  : never;
```
When we apply DistributiveOmit to our MusicProduct type, we get the anticipated result: a union of Album, CollectorEdition, and DigitalRelease with the id field omitted:
```
type MusicProductWithoutId = DistributiveOmit<MusicProduct, "id">;
              
type MusicProductWithoutId = Omit<Album, "id"> | Omit<CollectorEdition, "id"> | Omit<DigitalRelease, "id">
```

Structurally, this is the same as:

```
type MusicProductWithoutId =
  | {
      title: string;
      genre: string;
    }
  | {
      title: string;
      limitedEditionFeatures: string[];
    }
  | {
      title: string;
      digitalFormat: string;
    };
```
In situations where you need to use Omit with union types, using a distributive version will give you a much more predictable result.

For completeness, the DistributivePick type can be defined in a similar way:
```
type DistributivePick<T, K extends keyof T> = T extends any
  ? Pick<T, K>
  : never;
```
---


In [ ]:
// Types for getting the return type of an asynchronous function
type PromiseType<T> = T extends Promise<infer U> ? U : never

export type ErrorType = { error: string }

/**
 * Represents the return type of an asynchronous function.
 * It extracts the resolved value from a Promise and excludes any potential error type.
 * Used primarily to get supabase action return types.
 */
export type AsyncReturnType<T extends (...args: any) => Promise<any>> = Exclude<
  PromiseType<ReturnType<T>>,
  ErrorType
>

In [ ]:
// retreiving keys from a deeply nested object
// https://itnext.io/how-to-create-a-type-to-retrieve-all-keys-of-an-object-in-typescript-a5739d1c23e2

type DeepKeys<T> = T extends object
  ? {
      [K in keyof T]-?: K extends string | number
        ? `${K}` | `${K}.${DeepKeys<T[K]>}`
        : never;
    }[keyof T]
  : never;

---
API Helper functions

taken from: https://medium.com/the-syntax-diaries/your-typescript-apis-are-a-mess-heres-how-we-fixed-ours-c33908bb3e74

In [ ]:
// transform API responses into the correct format
function transformAPIUser(apiUser: ApiUser): User {
  return {
    id: apiUer.user_id,
    firstName: apiUser.first_name,
    settings: {
      theme: apiUser.settings.isDarkMode ? 'dark' : 'light',
      notifications: apiUser.settings.notificationsEnabled
    }
  }
}

// user type guards for runtime checks:
function isApiUser(data: unknown): data is ApiUser {
  if (!data || typeof data !== 'object') return false;

  const user = data as any;
  return (
    typeof user.user_id === 'number' &&
    typeof user.first_name === 'string' &&
    typeof settings === 'object'
  )
}

// call API using helpers
async function getUser(id: string): Promise<User> {
  const response = await fetch(`api/users/${id}`);
  const data = await response.json()

  if (!isApiUser(data)) {
    throw new Error('Invalid API Response')
  }

  return transformAPIUser(data)
}

// for more flexibility, use a generic type guard

function hasProperties<T extends object>( data: unknown, props: (keyof T)[]): data is Partial<T> {
  if (!data || typeof data !== 'object') return false; 
  return props.every(prop => prop in data);
}

// hasPropeties usage:
if (hasProperties<ApiUser>(DataTransfer, ['user_id', 'email'])) {
  // perform any ops since TS knows is has 'user_id' and 'email'
}

// use decorators to transfrom a response
function transformResponse<T,U>(
  guard: (data: unkown) => data is T,
  transformer: (data: t) => U
) {
  return function (
    target: any,
    propertyKey: string,
    descriptor: PropertyDescriptor
  ){
    const original = descriptor.value;

    descriptor.value = async function(...args: any[]) {
      const response = await original.apply(this, args);
      const data = await response.json();

      if (!guard(data)) {
        throw new Error('Invalid API Response' + propertyKey)
      }

      return transformer(data);
    }
  }
}

// decorator usage (needs a class)

class UserApi {
  @transformResponse(isApiUser, transformAPIUser)
  async getUser(id: string): Promise<User> {
    return fetch('/api/users/' + id)
  }
}
